In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 1

### Imports

Use `aimport` for `schedview` imports for ease of debugging.

In [ ]:
import warnings
import itertools
import panel as pn

In [ ]:
from astropy.time import Time, TimeDelta
import numpy as np
import pandas as pd

import bokeh
import bokeh.models

from astropy.time import Time

In [ ]:
from rubin_sim.scheduler.example import example_scheduler
from rubin_sim.scheduler import sim_runner
from rubin_sim.scheduler.model_observatory import ModelObservatory

In [ ]:
%aimport schedview
%aimport schedview.app.prenight
%aimport schedview.plot.nightbf

### Further preparation of the notebook

In [ ]:
pn.extension()

### Filter warnings

Several dependencies throw prodigious instances of (benign) warnings.
Suppress them to avoid poluting the executed notebook.

In [ ]:
warnings.filterwarnings(
    "ignore",
    module="astropy.time",
    message="Numerical value without unit or explicit format passed to TimeDelta, assuming days",
)
warnings.filterwarnings(
    "ignore",
    module="pandas",
    message="In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.",
)
warnings.filterwarnings(
    "ignore",
    module="healpy",
    message="divide by zero encountered in divide",
)
warnings.filterwarnings(
    "ignore",
    module="healpy",
    message="invalid value encountered in multiply",
)
warnings.filterwarnings(
    "ignore",
    module="holoviews",
    message="Discarding nonzero nanoseconds in conversion.",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim",
    message="invalid value encountered in arcsin",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim",
    message="All-NaN slice encountered",
)

## Run a simulation

For this example, simulate starting the default first day of observing:

In [ ]:
mjd_start = ModelObservatory().mjd_start

In [ ]:
survey_length = 1.0  # days
scheduler = example_scheduler()
scheduler.keep_rewards = True
observatory = ModelObservatory(mjd_start=mjd_start)
observatory, scheduler, observations, reward_df, obs_rewards = sim_runner(
    observatory,
    scheduler,
    mjd_start=mjd_start,
    survey_length=survey_length,
    record_rewards=True,
    filename="rewards.db",
)

In [ ]:
obs_night = Time(np.median(observations["mjd"]), format="mjd")

In [ ]:
reward_df

In [ ]:
def configure_survey_selector(survey_selector, reward_df, tier):
    surveys = (
        reward_df.set_index("tier_label").loc[tier, "survey_label"].unique().tolist()
    )
    survey_selector.options = surveys
    survey_selector.value = surveys[:10] if len(surveys) > 10 else surveys

In [ ]:
def configure_basis_function_selector(basis_function_selector, reward_df, tier):
    basis_functions = (
        reward_df.set_index("tier_label").loc[tier, "basis_function"].unique().tolist()
    )
    basis_function_selector.options = ["Total"] + basis_functions
    basis_function_selector.value = "Total"

In [ ]:
night = pn.widgets.DatePicker(name="Night", value=obs_night.datetime.date())
tier = pn.widgets.Select(
    name="Tier",
    options=reward_df.tier_label.unique().tolist(),
    value="tier 2",
    width_policy="fit",
)

surveys = pn.widgets.MultiSelect(
    name="Displayed surveys", options=["foo"], value=["foo"], width_policy="fit"
)
configure_survey_selector(surveys, reward_df, tier.value)


def survey_selector_update_callback(surveys, event):
    new_tier = event.new
    configure_survey_selector(surveys, reward_df, new_tier)


tier.link(surveys, {"value": survey_selector_update_callback})

basis_function = pn.widgets.Select(
    name="Reward (Total or basis function maximum)",
    options=["Total"],
    value="Total",
    width_policy="fit",
)
configure_basis_function_selector(basis_function, reward_df, tier.value)


def basis_function_selector_update_callback(basis_function, event):
    new_tier = event.new
    configure_basis_function_selector(basis_function, reward_df, new_tier)


tier.link(basis_function, {"value": basis_function_selector_update_callback})

reward_plot = pn.bind(
    schedview.plot.nightbf.plot_rewards,
    reward_df,
    tier,
    night,
    None,
    obs_rewards,
    surveys,
    basis_function,
    plot_kwargs={"width": 1024},
)


infeasible_plot = pn.bind(
    schedview.plot.nightbf.plot_infeasible,
    reward_df,
    tier,
    night,
    None,
    surveys,
    plot_kwargs={"width": 1024},
)

pn.Column(night, pn.Row(tier, surveys, basis_function), reward_plot, infeasible_plot)